In [1]:
!pip install transformers torch accelerate

Defaulting to user installation because normal site-packages is not writeable
  Using cached accelerate-1.12.0-py3-none-any.whl.metadata (19 kB)
Using cached accelerate-1.12.0-py3-none-any.whl (380 kB)

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from transformers import pipeline

/home/a/Desktop/rag/RAG/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load a text generation pipeline with a small model
generator1 = pipeline('text-generation', model='gpt2')
# DistilGPT-2 (меньше GPT-2, быстрее)
generator2 = pipeline('text-generation', model='distilgpt2')

# DialoGPT-small (для диалогов)
generator3 = pipeline('text-generation', model='microsoft/DialoGPT-small')



Loading weights: 100%|██████████| 149/149 [00:00<00:00, 1103.58it/s, Materializing param=transformer.wte.weight]            
The tied weights mapping and config for this model specifies to tie transformer.wte.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


In [4]:
# Generate text
prompt = "reccomend me a good road movie"
result = generator1(prompt, max_length=50, num_return_sequences=1)
print(result[0]['generated_text'])

Passing `generation_config` together with generation-related arguments=({'max_length', 'num_return_sequences'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


reccomend me a good road movie. "

The movie is titled "The Godfather of the Furious", and is scheduled to hit theaters on July 23, 2018.

The movie is written by Andrew Garfield and stars Tom Hardy, Amy Adams, and Michael Fassbender.


## Попробуем Llama 2

Ниже — способ использовать Llama 2 через llama.cpp для локальной работы с квантованной моделью (CPU, компактные файлы).

Внимание: Llama 2 7B; для работы нужен минимум ~4 ГБ RAM.

In [5]:
!pip install llama-cpp-python huggingface-hub


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
!pip install llama-cpp-python huggingface-hub



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


: 

In [ ]:
# Локально через llama.cpp
# Установите: pip install llama-cpp-python huggingface-hub

from huggingface_hub import hf_hub_download
from llama_cpp import Llama

# Download LLaMA-2 13B Chat GGUF model (Q4_K_M quantization)
model_path = hf_hub_download(
    repo_id="TheBloke/Llama-2-13B-Chat-GGUF",
    filename="llama-2-13b-chat.Q4_K_M.gguf"
)
print(f"Model downloaded to: {model_path}")

llm = Llama(
    model_path=model_path,
    chat_format="llama-2",
    n_ctx=2048,
    n_threads=8,   # можно поставить = числу физических ядер CPU
    verbose=False
)

messages = [{"role": "user", "content": "Recommend a good road movie."}]
resp = llm.create_chat_completion(messages=messages)

print(resp["choices"][0]["message"]["content"])

Model downloaded to: /home/a/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-Chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q4_K_M.gguf


llama_context: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized


In [7]:
# Локально через llama.cpp
# Установите: pip install llama-cpp-python huggingface-hub

from huggingface_hub import hf_hub_download
from llama_cpp import Llama

# Download Mistral-7B-Instruct GGUF model (Q4_K_M quantization)
model_path = hf_hub_download(
    repo_id="TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
    filename="mistral-7b-instruct-v0.1.Q4_K_M.gguf"
)
print(f"Model downloaded to: {model_path}")

llm = Llama(
    model_path=model_path,
    chat_format="mistral-instruct",
    n_ctx=2048,
    n_threads=8,   # можно поставить = числу физических ядер CPU
    verbose=False
)

messages = [{"role": "user", "content": "Recommend a good road movie."}]
resp = llm.create_chat_completion(messages=messages)

print(resp["choices"][0]["message"]["content"])

ModuleNotFoundError: No module named 'huggingface_hub'